In [128]:
#!pip install missingno
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import missingno as msno
from sklearn.tree import DecisionTreeClassifier

In [129]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# ? 데이터 제거 제출|

# 결측치 처리


In [130]:
# 3 Customer_care_calls 5423 non-null float64 - 0 으로 채움
train['Customer_care_calls'].fillna(0, inplace=True)
test['Customer_care_calls'].fillna(0, inplace=True)

In [131]:
# 6 Prior_purchases 6049 non-null float64 - 0 으로 채움
train['Prior_purchases'].fillna(0, inplace=True)
test['Prior_purchases'].fillna(0, inplace=True)

In [132]:
# 9 Discount_offered 3468 non-null float64 - 0 으로 채움
train['Discount_offered'].fillna(0, inplace=True)
test['Discount_offered'].fillna(0, inplace=True)

In [133]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   4000 non-null   int64  
 1   Warehouse_block      4000 non-null   object 
 2   Mode_of_Shipment     4000 non-null   object 
 3   Customer_care_calls  4000 non-null   float64
 4   Customer_rating      4000 non-null   int64  
 5   Cost_of_the_Product  4000 non-null   int64  
 6   Prior_purchases      4000 non-null   float64
 7   Product_importance   4000 non-null   object 
 8   Gender               4000 non-null   object 
 9   Discount_offered     4000 non-null   float64
 10  Weight_in_gms        4000 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 343.9+ KB


# 이상치 처리

In [202]:
# 'Warehouse_block' 컬럼 이름 수정
train.rename(columns={'Warehouse_Block': 'Warehouse_block'}, inplace=True)


In [134]:
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')

In [135]:
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')

In [136]:
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Roadzk', 'Road')


In [137]:
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Road', 'Road')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Ship', 'Ship')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flight', 'Flight')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Road', 'Road')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Ship', 'Ship')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flight', 'Flight')


In [138]:
test['Mode_of_Shipment'].unique()

array(['Ship', 'Flight', 'Road', '?'], dtype=object)

In [139]:
# Cost_of_the_Product 9999 이상치 제거 
train_filtered = train[train['Cost_of_the_Product'] != 9999]
# Cost_of_the_Product 9999 이상치 제거 
test_filtered = test[test['Cost_of_the_Product'] != 9999]

In [140]:
median_cost = train['Cost_of_the_Product'].median()
train['Cost_of_the_Product'].replace(9999, median_cost, inplace=True)
test['Cost_of_the_Product'].replace(9999, median_cost, inplace=True)

In [141]:
# 99를 평점 중앙값으로 대체
median_cost = train['Customer_rating'].median()
train['Customer_rating'].replace(99, median_cost, inplace=True)
test['Customer_rating'].replace(99, median_cost, inplace=True)

In [174]:
train['Product_importance'].replace('mediumm', 'medium', inplace=True)
train['Product_importance'].replace('loww', 'low', inplace=True)
train['Product_importance'].replace('highh', 'high', inplace=True)
test['Product_importance'].replace('mediumm', 'medium', inplace=True)
test['Product_importance'].replace('loww', 'low', inplace=True)
test['Product_importance'].replace('highh', 'high', inplace=True)

In [175]:
# 카테고리 이름 수정
test['Product_importance'].unique()

array(['high', 'medium', 'low', '?'], dtype=object)

In [144]:
# train 에서 필요없는 컬럼 제거
# 'ID' 컬럼 삭제

# 이미 삭제되어있다면 실행 안됨  
train.drop('ID', axis=1, inplace=True)
train.drop('Gender', axis=1, inplace=True)
test.drop('ID', axis=1, inplace=True)
test.drop('Gender', axis=1, inplace=True)

In [145]:
# Weight_in_gms 뺀 데이터
train.drop('Weight_in_gms', axis=1, inplace=True)
test.drop('Weight_in_gms', axis=1, inplace=True)

In [203]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  6999 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6999 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Discount_offered     6999 non-null   float64
 8   Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 492.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      4000 non-null   object 
 1   Mode_of_Shipment   

In [147]:
# 범주형 데이터만 뽑기
train.select_dtypes(include=['object'])

,Warehouse_block,Mode_of_Shipment,Product_importance
0,C,Ship,low
1,F,Flight,low
2,F,Ship,low
3,F,Ship,medium
4,D,Flight,high
...,...,...,...
6994,D,Ship,medium
6995,F,Ship,medium
6996,D,Ship,low
6997,A,Road,medium


In [204]:
# 원핫 인코딩
X_train = pd.get_dummies(train)
X_test = pd.get_dummies(test)

In [205]:
X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer_care_calls        6999 non-null   float64
 1   Customer_rating            6999 non-null   int64  
 2   Cost_of_the_Product        6999 non-null   int64  
 3   Prior_purchases            6999 non-null   float64
 4   Discount_offered           6999 non-null   float64
 5   Reached.on.Time_Y.N        6999 non-null   int64  
 6   Warehouse_block_A          6999 non-null   uint8  
 7   Warehouse_block_B          6999 non-null   uint8  
 8   Warehouse_block_C          6999 non-null   uint8  
 9   Warehouse_block_D          6999 non-null   uint8  
 10  Warehouse_block_F          6999 non-null   uint8  
 11  Mode_of_Shipment_?         6999 non-null   uint8  
 12  Mode_of_Shipment_Flight    6999 non-null   uint8  
 13  Mode_of_Shipment_Road      6999 non-null   uint8

In [207]:
set(X_train.columns) - set(X_test.columns)

{'Reached.on.Time_Y.N'}

In [191]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer_care_calls        6999 non-null   float64
 1   Customer_rating            6999 non-null   int64  
 2   Cost_of_the_Product        6999 non-null   int64  
 3   Prior_purchases            6999 non-null   float64
 4   Discount_offered           6999 non-null   float64
 5   Reached.on.Time_Y.N        6999 non-null   int64  
 6   Warehouse_Block_A          6999 non-null   uint8  
 7   Warehouse_Block_B          6999 non-null   uint8  
 8   Warehouse_Block_C          6999 non-null   uint8  
 9   Warehouse_Block_D          6999 non-null   uint8  
 10  Warehouse_Block_F          6999 non-null   uint8  
 11  Mode_of_Shipment_?         6999 non-null   uint8  
 12  Mode_of_Shipment_Flight    6999 non-null   uint8  
 13  Mode_of_Shipment_Road      6999 non-null   uint8

In [192]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 6999 entries, 0 to 6998
Series name: Reached.on.Time_Y.N
Non-Null Count  Dtype
--------------  -----
6999 non-null   int64
dtypes: int64(1)
memory usage: 54.8 KB


In [153]:
tree_model = DecisionTreeClassifier()


In [154]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer_care_calls        6999 non-null   float64
 1   Customer_rating            6999 non-null   int64  
 2   Cost_of_the_Product        6999 non-null   int64  
 3   Prior_purchases            6999 non-null   float64
 4   Discount_offered           6999 non-null   float64
 5   Reached.on.Time_Y.N        6999 non-null   int64  
 6   Warehouse_block _A         6999 non-null   uint8  
 7   Warehouse_block _B         6999 non-null   uint8  
 8   Warehouse_block _C         6999 non-null   uint8  
 9   Warehouse_block _D         6999 non-null   uint8  
 10  Warehouse_block _F         6999 non-null   uint8  
 11  Mode_of_Shipment_?         6999 non-null   uint8  
 12  Mode_of_Shipment_Flight    6999 non-null   uint8  
 13  Mode_of_Shipment_Road      6999 non-null   uint8

In [159]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 6999 entries, 0 to 6998
Series name: Reached.on.Time_Y.N
Non-Null Count  Dtype
--------------  -----
6999 non-null   int64
dtypes: int64(1)
memory usage: 54.8 KB


In [208]:
y_train= X_train['Reached.on.Time_Y.N']
X_train = X_train.drop('Reached.on.Time_Y.N', axis=1)


In [1]:
tree_model.fit(X_train, y_train)
DecisionTreeClassifier()

NameError: name 'tree_model' is not defined

In [210]:
tree_model.score(X_train,y_train)

0.9988569795685098

In [586]:
tree_model1 = DecisionTreeClassifier(max_leaf_nodes=4,max_depth=8)

In [587]:
tree_model1

DecisionTreeClassifier(max_depth=8, max_leaf_nodes=4)

In [588]:
tree_model1.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, max_leaf_nodes=4)

In [589]:
tree_model1.score(X_train, y_train)

0.6196599514216317

In [592]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(tree_model1, X_train, y_train, cv=5)
np.mean(score)

0.6192336362708056

In [593]:
test.info

<bound method DataFrame.info of      Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0                  F             Ship                  3.0                1   
1                  F           Flight                  3.0                4   
2                  A             Ship                  0.0                5   
3                  C             Ship                  0.0                1   
4                  F           Flight                  4.0                2   
...              ...              ...                  ...              ...   
3995               A             Ship                  4.0                1   
3996               C             Road                  0.0                3   
3997               B           Flight                  0.0                3   
3998               C             Ship                  6.0                4   
3999               D             Ship                  5.0                1   

      Cost_of_the_P

In [594]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      4000 non-null   object 
 1   Mode_of_Shipment     4000 non-null   object 
 2   Customer_care_calls  4000 non-null   float64
 3   Customer_rating      4000 non-null   int64  
 4   Cost_of_the_Product  4000 non-null   int64  
 5   Prior_purchases      4000 non-null   float64
 6   Product_importance   4000 non-null   object 
 7   Discount_offered     4000 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 250.1+ KB


In [595]:
pre = tree_model1.predict(X_test)

In [596]:
pre

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [597]:
result = pd.read_csv('./sampleSubmission.csv')

In [598]:
len(pre)


4000

In [599]:
result['Reached.on.Time_Y.N']=pre
result.to_csv('예측답_2번.csv', index=False)